In [8]:
import numpy as np  # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd  # data processing

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.feature_selection import SelectKBest,chi2
# from sklearn.tree import DecisionTreeClassifier

In [10]:
df = pd.read_csv('./reference/100_day_ml_source_code/day29-sklearn-pipelines/train.csv')
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [339]:
from sklearn.model_selection import train_test_split

x = df.loc[:, df.columns != 'Survived']
y = df.loc[:, ['Survived']]

x, y, x_test, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x.info()
# x

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 331 to 102
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


In [340]:
print("'Age' column index: ", x.columns.get_loc('Age'))
print("'Embarked' column index: ", x.columns.get_loc('Embarked'))
print(x.isna().sum())

'Age' column index:  2
'Embarked' column index:  6
Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [341]:
transformers = [
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
]
t1 = ColumnTransformer(transformers=transformers, remainder='passthrough')

In [315]:
# x.iloc[:, [2, 6]]

In [361]:
df1 = t1.fit_transform(x)
df1

array([[45.5, 'S', 1, ..., 0, 0, 28.5],
       [23.0, 'S', 2, ..., 0, 0, 13.0],
       [32.0, 'S', 3, ..., 0, 0, 7.925],
       ...,
       [41.0, 'S', 3, ..., 2, 0, 14.1083],
       [14.0, 'S', 1, ..., 1, 2, 120.0],
       [21.0, 'S', 1, ..., 0, 1, 77.2875]], dtype=object)

In [428]:
print("'Sex' column index: ", x.columns.get_loc('Sex'))

t2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough')
# t2

'Sex' column index:  1


In [435]:
# OneHotEncoder(handle_unknown='ignore').fit_transform(df2[:, [1]]).toarray()
x.iloc[:, [1, 6]]

,Sex,Embarked
331,male,S
733,male,S
382,male,S
704,male,S
813,female,S
...,...,...
106,female,S
270,male,S
860,male,S
435,female,S


In [438]:
df2 = t2.fit_transform(x)
df2

array([[  0.    ,   1.    ,   0.    , ...,   0.    ,   0.    ,  28.5   ],
       [  0.    ,   1.    ,   0.    , ...,   0.    ,   0.    ,  13.    ],
       [  0.    ,   1.    ,   0.    , ...,   0.    ,   0.    ,   7.925 ],
       ...,
       [  0.    ,   1.    ,   0.    , ...,   2.    ,   0.    ,  14.1083],
       [  1.    ,   0.    ,   0.    , ...,   1.    ,   2.    , 120.    ],
       [  0.    ,   1.    ,   0.    , ...,   0.    ,   1.    ,  77.2875]])

In [439]:
t3 = ColumnTransformer(transformers=[
    ('scaler', MinMaxScaler(), slice(0, 10))
], remainder='passthrough')

In [422]:
df3 = t3.fit_transform(df2)
df3

array([[0.0, 0.0, 1.0, ..., 'male', 0, 0],
       [0.0, 0.0, 1.0, ..., 'male', 0, 0],
       [0.0, 0.0, 1.0, ..., 'male', 0, 0],
       ...,
       [0.0, 0.0, 1.0, ..., 'male', 2, 0],
       [0.0, 0.0, 1.0, ..., 'female', 1, 2],
       [0.0, 0.0, 1.0, ..., 'male', 0, 1]], dtype=object)

In [459]:
pipe = Pipeline([
    ('imputes', t1),
    ('ohe', t2),
    ('scalars', t3)
])

# pipe.fit(x)
# pipe.transform(x)

# Alternate Syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)